# Challenge 9: Poisson-GLM Challenges

| Title        | Answer                     |
| :---------- | :------------------------ |
| Topic:       | Poisson-GLM Challenges | 
| Date:        | 2016/08/06                 |
| Name:        | Michelle L. Gill           |
| Worked with: | None                       | 




In [38]:
import pandas as pd
import numpy as np

from pandas.io.stata import StataReader
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.cross_validation import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error

from scipy.stats import chisqprob, chi2

## Question 1

Model the damage incident counts with a Poisson Regression.

Hint: You can look at the previous ipython notebook with the logistic GLM example to see how you can do GLM with statsmodels
Remember that you will have to create dummy variables for categorical variables, and if you have time bins (like "1960-1964"), you have the option of either a) treating each bin as a category (and create dummies for each bin), or b) treat it as a continuous variable and take the mid-value (1962). Also remember to add a constant (to model the intercept).

Take a look at the statsmodels summary table, the goodness of fit indicators (Deviance, Pearson's chi square statistic) and the coefficients. Is this a good model?

In [20]:
! curl http://data.princeton.edu/wws509/datasets/ships.dta -o ships.dta
ships = StataReader('ships.dta').read()
ships.head(1)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2324  100  2324    0     0   5504      0 --:--:-- --:--:-- --:--:--  103k


,type,construction,operation,months,damage
0,A,1960-64,1960-74,127.0,0.0


In [25]:
# Convert date ranges to means
ships['construction_mean'] = (ships
                              .construction
                              .str.extract(r"""([0-9]{2})-([0-9]{2})""", expand=True)
                              .astype(int)
                              .mean(axis=1))

ships['operation_mean'] = (ships
                              .operation
                              .str.extract(r"""([0-9]{2})-([0-9]{2})""", expand=True)
                              .astype(int)
                              .mean(axis=1))

In [26]:
Y, X = dmatrices("damage ~ type + construction_mean + operation_mean + months", 
                 data=ships, return_type='dataframe')

poisson_model1 = sm.GLM(Y, X, family=sm.families.Poisson())

poisson_model1.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 damage   No. Observations:                   34
Model:                            GLM   Df Residuals:                       26
Model Family:                 Poisson   Df Model:                            7
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -108.37
Date:                Mon, 08 Aug 2016   Deviance:                       118.88
Time:                        17:15:50   Pearson chi2:                     112.
No. Iterations:                     9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Intercept            -6.2302      1.426     -4.370      0.000        -9.025    -3.436
type[T.B]             0.9475      0.206      4.598      0.000         0.544     1.351
type[T.C]            -1.2102      0.327     -3.696      0.000        -1.852    -0.568
type[T.D]            -0.8642      0.288     -3.006      0.003        -1.428    -0.301
type[T.E]            -0.1139      0.235     -0.485      0.628        -0.575     0.347
construction_mean     0.0564      0.014      4.017      0.000         0.029     0.084
operation_mean        0.0546      0.014      3.934      0.000         0.027     0.082
months              4.89e-05   7.11e-06      6.875      0.000       3.5e-05  6.28e-05
=====================================================================================
"""

Based on coefficient ranges and fit parameters, this does not appear to be a good model.

## Question 2

The months of service provides the time interval in which a ship has chances to acquire damages. It can be thought of "exposure", and this column can be used as an offset.

You can check these two resources for a quick idea on exposure and using an offset:

* Offset in Wikipedia
* When to use an offset in CrossValidated (StackOverflow for statistics)

Try your model with months of service as the offset. Does it perform better?

In [27]:
Y, X = dmatrices("damage ~ type + construction_mean + operation_mean", 
                 data=ships, return_type='dataframe')

poisson_model2 = sm.GLM(Y, X, 
                        offset=np.log(ships.months),
                        family=sm.families.Poisson())

In [28]:
poisson_model2.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 damage   No. Observations:                   34
Model:                            GLM   Df Residuals:                       27
Model Family:                 Poisson   Df Model:                            6
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -78.076
Date:                Mon, 08 Aug 2016   Deviance:                       58.286
Time:                        17:15:51   Pearson chi2:                     64.6
No. Iterations:                     9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Intercept           -11.2451      1.031    -10.911      0.000       -13.265    -9.225
type[T.B]            -0.5397      0.178     -3.031      0.002        -0.889    -0.191
type[T.C]            -0.6270      0.329     -1.903      0.057        -1.273     0.019
type[T.D]            -0.2328      0.288     -0.809      0.419        -0.797     0.332
type[T.E]             0.4071      0.235      1.733      0.083        -0.053     0.868
construction_mean     0.0445      0.013      3.450      0.001         0.019     0.070
operation_mean        0.0349      0.012      2.880      0.004         0.011     0.059
=====================================================================================
"""

Note that the log-likelihood is might higher with this model and the deviance is much smaller.

## Question 3

Now separate your data (even though it's only 14 rows) into a training and test set (your test will only be 4 or 5 rows), and check if you predict well (you can look at mean absolute error or mean squared error using sklearn.metrics).

In [48]:
train, test = train_test_split(ships, test_size=0.25, random_state=4444)

print train.shape, test.shape

Y_train, X_train = dmatrices("damage ~ type + construction_mean + operation_mean", 
                             data=train, return_type='dataframe')


Y_test,  X_test =  dmatrices("damage ~ type + construction_mean + operation_mean", 
                             data=test, return_type='dataframe')

poisson_model3 = sm.GLM(Y_train, X_train, 
                        offset=np.log(train['months']),
                        family=sm.families.Poisson()).fit()

(25, 7) (9, 7)


In [49]:
Y_pred = poisson_model3.predict(X_test)

In [50]:
mean_absolute_error(Y_test, Y_pred)

5.4412987707392837

In [51]:
mean_squared_error(Y_test, Y_pred)

62.280577572572106

## Question 4

Deviance. Compute the difference in Deviance statistics for your model and the null model. This is called the null deviance. You can do this in one of 2 ways:

We need the deviance for the null model (a model where none of the explanatory variables are used; it's just a model with a mean guess). To do that, fit a poisson regression with only a constant. Get the deviance for this null model. Take the difference of deviances between your model and this null model.

Use statsmodels.genmod.generalized_linear_model.GLMResults
Check if this difference is extreme enough that we can reject the null hypothesis. If we can't reject the null hypothesis, we cannot say that this model tells us more than that trivial, null model. To calculate the p-value (prob. of getting a deviance difference at least as extreme as this under the null hypothesis), we need to do a hypothesis test.

Is your model better than the null model?

In [52]:
Y_train, X_train = dmatrices("damage ~ 1", 
                             data=train, return_type='dataframe')

Y_test,  X_test  = dmatrices("damage ~ 1", 
                             data=test, return_type='dataframe')

poisson_model_null = sm.GLM(Y_train, X_train, 
                            #offset=np.log(train.months),
                            family=sm.families.Poisson())

In [53]:
poisson_model_null.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 damage   No. Observations:                   25
Model:                            GLM   Df Residuals:                       24
Model Family:                 Poisson   Df Model:                            0
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -300.47
Date:                Mon, 08 Aug 2016   Deviance:                       528.71
Time:                        17:46:04   Pearson chi2:                     616.
No. Iterations:                     8                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      2.5080      0.057     43.943      0.000         2.396     2.620
==============================================================================
"""

## Question 5
Now, instead of a poisson regression, do an ordinary least squares regression with log Y. Compare the models. Are the coefficients close? Do they perform similarly?

In [46]:
Y_train, X_train = dmatrices("np.log(damage+0.1) ~ type + construction_mean + operation_mean + months", 
                             data=train, return_type='dataframe')


Y_test, X_test = dmatrices("np.log(damage+0.1) ~ type + construction_mean + operation_mean + months", 
                             data=test, return_type='dataframe')

linear_model = sm.OLS(Y_train, X_train).fit()

In [47]:
linear_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     np.log(damage + 0.1)   R-squared:                       0.769
Model:                              OLS   Adj. R-squared:                  0.673
Method:                   Least Squares   F-statistic:                     8.072
Date:                  Mon, 08 Aug 2016   Prob (F-statistic):           0.000222
Time:                          17:34:30   Log-Likelihood:                -37.666
No. Observations:                    25   AIC:                             91.33
Df Residuals:                        17   BIC:                             101.1
Df Model:                             7                                         
Covariance Type:              nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Intercept           -17.4152      4.944     -3.523      0.003       -27.846    -6.984
type[T.B]             1.5305      1.191      1.285      0.216        -0.983     4.044
type[T.C]            -0.7477      0.856     -0.873      0.395        -2.555     1.059
type[T.D]            -0.9369      0.770     -1.216      0.241        -2.562     0.688
type[T.E]            -0.6235      0.942     -0.662      0.517        -2.612     1.365
construction_mean     0.2587      0.057      4.537      0.000         0.138     0.379
operation_mean        0.0015      0.057      0.027      0.979        -0.118     0.121
months             8.948e-05   4.52e-05      1.981      0.064     -5.83e-06     0.000
==============================================================================
Omnibus:                        0.404   Durbin-Watson:                   1.841
Prob(Omnibus):                  0.817   Jarque-Bera (JB):                0.507
Skew:                           0.252   Prob(JB):                        0.776
Kurtosis:                       2.517   Cond. No.                     2.30e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.3e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

All coefficients are important in the model, which is a tell-tale sign that the fit is poor.

## Question 6
Now, let's do this on another dataset: Smoking and Cancer. This dataset has information on lung cancer deaths by age and smoking status. It has these columns:

age: in five-year age groups coded 1 to 9 for 40-44, 45-49, 50-54, 55-59, 60-64, 65-69, 70-74, 75-79, 80+.
smoking status: coded 1 = doesn't smoke, 2 = smokes cigars or pipe only, 3 = smokes cigarrettes and cigar or pipe, and 4 = smokes cigarrettes only,
population: in hundreds of thousands
deaths: number of lung cancer deaths in a year.
That population looks a lot like an offset!

Fit poisson and OLS models and compare them.

In [54]:
! curl http://data.princeton.edu/wws509/datasets/smoking.dta -o smoking_cancer.dta
smoking = StataReader('smoking_cancer.dta').read()
smoking.head(1)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1840  100  1840    0     0   3988      0 --:--:-- --:--:-- --:--:-- 63448


,age,smoke,pop,dead
0,40-44,Doesn't smoke,656.0,18.0


In [69]:
# Convert age ranges to means
smoking['age_mean'] = (smoking
                      .age
                      .str.extract(r"""([0-9]{2})(?:-([0-9]{2}))?""", expand=True)
                      .astype(float)
                      .mean(axis=1))